In [ ]:
%%python
import findspark
import os

findspark.init("/home/ubuntu/spark-3.0.0-bin-hadoop2.7")

In [ ]:
%%init_spark
launcher.num_executors = 16
launcher.executor_cores = 24
launcher.executor_memory = '10g'
launcher.conf.set("spark.app.name", "Generate hibench parquet")
launcher.conf.set("spark.authenticate", "false")
launcher.conf.set("spark.deploy-mode", "client")
launcher.conf.set("spark.task.cpus", "4")
launcher.conf.set("spark.master", "yarn")


In [ ]:
import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.ml.feature.{LabeledPoint => NewLabeledPoint}
import org.apache.spark.sql.functions._
import org.apache.spark.ml._


In [ ]:
spark.sparkContext.getConf.getAll.foreach(println)

In [ ]:
println(spark.sparkContext.applicationId)

In [ ]:
%%python
print(spark.sparkContext.applicationId)

In [ ]:
sc.defaultParallelism

In [ ]:
println(s"Load and parse the data file")
val mllibRDD: RDD[LabeledPoint] = spark.sparkContext.objectFile("hdfs:///HiBench/XGBoost/Input")
// Convert to ML LabeledPoint and to DataFrame
val mlRDD: RDD[NewLabeledPoint] = mllibRDD.map { p => NewLabeledPoint(p.label, p.features.asML) }
val data = mlRDD.toDF("label", "features").coalesce(384)

val vecToArray = udf( (xs: linalg.Vector) => xs.toArray )
val dfArr = data.withColumn("featuresArr" , vecToArray($"features") )
val feats = Array(
    "f0", "f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9",
    "f10", "f11", "f12", "f13", "f14", "f15", "f16", "f17", "f18", "f19",
    "f20", "f21", "f22", "f23", "f24", "f25", "f26", "f27", "f28", "f29",
    "f30", "f31", "f32", "f33", "f34", "f35", "f36", "f37", "f38", "f39",
    "f40", "f41", "f42", "f43", "f44", "f45", "f46", "f47", "f48", "f49"
)
val sqlExpr = feats.zipWithIndex.map{ case (alias, idx) => col("featuresArr").getItem(idx).cast("float").as(alias) }
val ldf = dfArr.select(sqlExpr : _*).withColumn("id", monotonicallyIncreasingId)
val rdf = dfArr.select(col("label")).withColumn("id", monotonicallyIncreasingId)
val df = ldf.join(rdf, "id").drop("id")
df.write.mode("overwrite").parquet("hdfs:///HiBench600Mx50.dataframe.float.parquet")

df.printSchema();
println(data.count())


In [ ]:
%%python

# exit gracefully
spark.stop()